In [1]:

# Nifty Stock Price Prediction with LSTM

# Introduction:
# This project uses a Long Short-Term Memory (LSTM) neural network model to predict the Nifty50 stock index's closing prices. LSTM is a type of recurrent neural network (RNN) which is particularly suited for sequence prediction problems.

# Data:
# The dataset consists of historical Nifty50 index prices, fetched using the yfinance library, spanning from January 1, 2022, to August 18, 2023.

# Features:

# Date: The date of recorded data.
# Close: The closing price of Nifty50 for the respective date.
# SMA_30: 30-day Simple Moving Average of the closing prices.
# EMA_30: 30-day Exponential Moving Average of the closing prices.
# Methodology:

# Data Collection:
# Using the yfinance library, we fetch historical data of the Nifty50 index. This provides us with daily open, high, low, close, adjusted close, volume, and dividends split information. We then calculate and append 30-day SMA and EMA for our analysis.

# Data Preprocessing:

# Scaling: We scale the closing price data between 0 and 1 using MinMaxScaler from Scikit-learn. Scaling is essential when working with neural network architectures.
# Training and Test Split: We reserve 75% of our data for training and the remainder for testing.
# Sequence Creation: For time series forecasting with LSTMs, the data needs to be structured as samples, where each sample is a time step with one or multiple observations. We create sequences of 30 days for training our LSTM model.
# Model Building:

# LSTM Layers: We use two LSTM layers with 50 units each. LSTMs can remember patterns over long sequences and are less susceptible to the vanishing gradient problem compared to vanilla RNNs.
# Dropout: Dropout layers are added after each LSTM layer to prevent overfitting. They randomly drop specified proportions of neurons during training.
# Dense Layers: Two dense layers transform LSTM outputs to our desired shape.
# Training:

# Optimizer: Adam optimizer is used because of its adaptive learning rates and efficient computation.
# Loss Function: Mean Squared Error (MSE) as it's a regression problem.
# Epochs and Batch Size: We train the model for 200 epochs with a batch size of 50.
# Validation:

# Overfitting Indicator: A custom function checks if there's a notable difference between training and validation loss, signaling possible overfitting.
# Evaluation Metrics: Root Mean Square Error (RMSE) gives an idea about the magnitude of error our model makes in its predictions. R^2 (or R-squared) provides a measure of how well future samples are likely to be predicted by the model.
# Visualization:
# Using plotly, we visualize:

# Historical and predicted closing prices.
# Moving averages: SMA_30 & EMA_30 for trend analysis.
# Predictions for future days.
# Results:
# The model provides predictions on the test dataset, which can be visualized alongside the actual data and moving averages. Moreover, it offers a forecast for future dates, which can be helpful for investment and analysis purposes.

# Conclusion & Future Work:
# The LSTM model provides a powerful tool for stock price prediction. However, stock prices are influenced by a myriad of unpredictable factors, making them inherently challenging to forecast accurately consistently.

# For future improvements:

# Incorporating additional data like news sentiments, other technical indicators, or even global economic indicators could enhance the model.
# Trying out different architectures or models, such as GRU or Transformer-based models.
# Hyperparameter tuning using libraries like Optuna or Keras Tuner can help in achieving a better model fit.
# Note: It's essential to understand that stock markets are influenced by a plethora of unpredictable factors, and one should not solely rely on predictive models for investment decisions.

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

# User input for the start date and number of future days
start_date = input("Enter the start date in the format YYYY-MM-DD: ")
number_of_future_days = int(input("Enter the number of future days you want to predict: "))

end_date = datetime.now().strftime('%Y-%m-%d')  # Set end date as today's date

# Fetch Nifty historical data
def get_data():
    df = yf.download('^NSEI', start=start_date, end=end_date)
    return df

# Check for overfitting
def is_overfitting(train_loss, val_loss):
    if abs(train_loss[-1] - val_loss[-1]) > abs(train_loss[0] - val_loss[0]):
        return True
    return False

# Function for future predictions
def predict_future_days(model, last_days_scaled):
    future_predictions = []
    for _ in range(number_of_future_days):
        prediction = model.predict(last_days_scaled.reshape(1, 30, 1))
        future_predictions.append(prediction[0, 0])
        last_days_scaled = np.append(last_days_scaled[1:], prediction)
    return scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

df = get_data()
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset) * 0.75)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []

for i in range(30, len(train_data)):
    x_train.append(train_data[i-30:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(x_train, y_train, batch_size=50, epochs=200, validation_split=0.2)

if is_overfitting(history.history['loss'], history.history['val_loss']):
    print("The model appears to be overfitting.")
else:
    print("The model does not appear to be overfitting.")

test_data = scaled_data[training_data_len - 30:, :]
x_test = []
y_test = dataset[training_data_len:, :]

for i in range(30, len(test_data)):
    x_test.append(test_data[i-30:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)
print(f"RMSE: {rmse:.2f}")
print(f"R^2 Score: {r2:.2f}")

last_30_days_scaled = scaled_data[-30:]
predictions_future = predict_future_days(model, last_30_days_scaled)

train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train['Close'], mode='lines', name='Train Data'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Close'], mode='lines', name='True Data'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Predictions'], mode='lines', name='Predictions'))

future_dates = pd.date_range(start=df.index[-1], periods=number_of_future_days+1)[1:]
future_df = pd.DataFrame(predictions_future, index=future_dates, columns=['Predictions'])
fig.add_trace(go.Scatter(x=future_df.index, y=future_df['Predictions'], mode='lines', name='Future Predictions', line=dict(dash='dot')))

fig.update_layout(
    title='Nifty Close Price Predictions',
    xaxis_title='Date',
    yaxis_title='Close Price',
    xaxis_rangeslider_visible=True
)

fig.show()


Enter the start date in the format YYYY-MM-DD: 2020-09-09
Enter the number of future days you want to predict: 20
[*********************100%%**********************]  1 of 1 completed
Epoch 1/200
9/9 [==============================] - 4s 104ms/step - loss: 0.1070 - val_loss: 0.0333
Epoch 2/200
9/9 [==============================] - 0s 31ms/step - loss: 0.0178 - val_loss: 0.0394
Epoch 3/200
9/9 [==============================] - 0s 48ms/step - loss: 0.0135 - val_loss: 0.0015
Epoch 4/200
9/9 [==============================] - 0s 41ms/step - loss: 0.0102 - val_loss: 0.0046
Epoch 5/200
9/9 [==============================] - 0s 41ms/step - loss: 0.0067 - val_loss: 0.0070
Epoch 6/200
9/9 [==============================] - 0s 43ms/step - loss: 0.0073 - val_loss: 0.0021
Epoch 7/200
9/9 [==============================] - 0s 39ms/step - loss: 0.0055 - val_loss: 0.0038
Epoch 8/200
9/9 [==============================] - 0s 27ms/step - loss: 0.0052 - val_loss: 0.0028
Epoch 9/200
9/9 [===============

<ipython-input-2-9fbb96ab7e3c>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions
